In [207]:
import pandas as pd
import numpy as np
import os
import re

In [208]:
df_nba_personal = pd.read_csv('raw/nba_personal.csv')
df_nba_stats = pd.read_csv('raw/nba_stats.csv')
df_nba_experience = pd.read_json('raw/nba_experience.json',lines=True)
df_nba_season = pd.read_excel('raw/nba_season.xlsx')

In [209]:
df_nba_personal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8162 entries, 0 to 8161
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   playerId                   8162 non-null   int64 
 1   playerSlug                 8162 non-null   object
 2   college                    6750 non-null   object
 3   draftinfo                  6977 non-null   object
 4   htwt                       2304 non-null   object
 5   name_birthdate_birthplace  8162 non-null   object
dtypes: int64(1), object(5)
memory usage: 382.7+ KB


In [210]:
df_nba_personal.head()

,playerId,playerSlug,college,draftinfo,htwt,name_birthdate_birthplace
0,366,allen-iverson,Georgetown,"1996: Rd 1, Pk 1 (PHI)",NaN,"Allen Iverson, 6/7/1975, Hampton, VA"
1,366,allen-iverson,Georgetown,"1996: Rd 1, Pk 1 (PHI)",NaN,"Allen Iverson, 6/7/1975, Hampton, VA"
2,366,allen-iverson,Georgetown,"1996: Rd 1, Pk 1 (PHI)",NaN,"Allen Iverson, 6/7/1975, Hampton, VA"
3,366,allen-iverson,Georgetown,"1996: Rd 1, Pk 1 (PHI)",NaN,"Allen Iverson, 6/7/1975, Hampton, VA"
4,366,allen-iverson,Georgetown,"1996: Rd 1, Pk 1 (PHI)",NaN,"Allen Iverson, 6/7/1975, Hampton, VA"


In [211]:
df_nba_personal['name'] = df_nba_personal['name_birthdate_birthplace'].apply(
    lambda x: x.split(',')[0].strip())

df_nba_personal['birthdate'] = df_nba_personal['name_birthdate_birthplace'].apply(
    lambda x: x.split(',')[1].strip())

df_nba_personal['birthplace'] = df_nba_personal['name_birthdate_birthplace'].apply(
    lambda x: x.split(',')[2].strip())

df_nba_personal['height'] = df_nba_personal['htwt'].apply(
    lambda x: x.split(',')[0].strip() if pd.notnull(x) else x)

df_nba_personal['weight'] = df_nba_personal['htwt'].apply(
    lambda x: x.split(',')[1].strip().split(' ')[0] if pd.notnull(x) else x)

df_nba_personal = df_nba_personal.drop(
    ['name_birthdate_birthplace', 'htwt'], axis=1)

In [212]:
df_nba_personal.head()

,playerId,playerSlug,college,draftinfo,name,birthdate,birthplace,height,weight
0,366,allen-iverson,Georgetown,"1996: Rd 1, Pk 1 (PHI)",Allen Iverson,6/7/1975,Hampton,NaN,NaN
1,366,allen-iverson,Georgetown,"1996: Rd 1, Pk 1 (PHI)",Allen Iverson,6/7/1975,Hampton,NaN,NaN
2,366,allen-iverson,Georgetown,"1996: Rd 1, Pk 1 (PHI)",Allen Iverson,6/7/1975,Hampton,NaN,NaN
3,366,allen-iverson,Georgetown,"1996: Rd 1, Pk 1 (PHI)",Allen Iverson,6/7/1975,Hampton,NaN,NaN
4,366,allen-iverson,Georgetown,"1996: Rd 1, Pk 1 (PHI)",Allen Iverson,6/7/1975,Hampton,NaN,NaN


In [213]:
df_nba_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8164 entries, 0 to 8163
Data columns (total 39 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   playerId                          8164 non-null   object 
 1   gamesPlayed                       8160 non-null   float64
 2   avgMinutes                        8160 non-null   object 
 3   avgFouls                          8164 non-null   object 
 4   flagrantFouls                     8164 non-null   object 
 5   technicalFouls                    8164 non-null   float64
 6   ejections                         8164 non-null   float64
 7   doubleDouble                      8164 non-null   float64
 8   tripleDouble                      8164 non-null   float64
 9   minutes                           8164 non-null   float64
 10  rebounds                          8164 non-null   float64
 11  fouls                             8164 non-null   float64
 12  avgReb

In [214]:
df_nba_stats.head()

,playerId,gamesPlayed,avgMinutes,avgFouls,flagrantFouls,technicalFouls,ejections,doubleDouble,tripleDouble,minutes,...,threePointFieldGoalsAttempted,freeThrowsMade,freeThrowsAttempted,assists,turnovers,avgSteals,avgBlocks,steals,blocks,totalPoints
0,366,NaN,NaN,2.0416667,0.0,5.0,0.0,4.0,0.0,1990.0,...,199.0,356.0,474.0,223.0,167.0,2.291667,0.145833,110.0,7.0,994.999968
1,366,NaN,NaN,2.3142858,0.0,12.0,0.0,1.0,0.0,2853.0,...,261.0,442.0,620.0,328.0,230.0,2.057143,0.071429,144.0,5.0,99999.000000
2,366,NaN,NaN,2.0704226,0.0,18.0,0.0,4.0,0.0,2979.0,...,306.0,585.0,719.0,325.0,237.0,2.507042,0.281690,178.0,20.0,NaN
3,366,60.0,NaN,dois,0.0,12.0,1.0,4.0,1.0,2622.0,...,268.0,475.0,585.0,331.0,237.0,2.800000,0.216667,168.0,13.0,NaN
4,366,82.0,2 3142858,"1,8292683",zero,8.0,0.0,6.0,0.0,3485.0,...,303.0,570.0,736.0,454.0,286.0,2.743902,0.158537,225.0,13.0,NaN


In [215]:
for idx,avgMinutes in enumerate(df_nba_stats["avgMinutes"]):
    if(type(avgMinutes) is str):
        df_nba_stats.loc[idx, 'avgMinutes'] = avgMinutes.replace(',', '.')
        df_nba_stats.loc[idx, 'avgMinutes'] = avgMinutes.replace(' ', '.')

        
for idx, avgFouls in enumerate(df_nba_stats["avgFouls"]):
    if (type(avgFouls) is str):
        df_nba_stats.loc[idx, 'avgFouls'] = avgFouls.replace(',','.')
        if avgFouls == 'dois':
            df_nba_stats.loc[idx, 'avgFouls'] = '2'
            
for idx, flagrantFouls in enumerate(df_nba_stats["flagrantFouls"]):
    if (type(flagrantFouls) is str):
        df_nba_stats.loc[idx, 'flagrantFouls'] = flagrantFouls.replace(
            ',', '.')
        if flagrantFouls == 'zero':
            df_nba_stats.loc[idx, 'flagrantFouls'] = '0'

for idx, playerId in enumerate(df_nba_stats["playerId"]):
    if (type(playerId) is str):
        if playerId == 'threehundred66':
            df_nba_stats.loc[idx, 'playerId'] = '366'
        
        
df_nba_stats["avgMinutes"] = df_nba_stats["avgMinutes"].astype(float)
df_nba_stats["avgFouls"] = df_nba_stats["avgFouls"].astype(float)
df_nba_stats["flagrantFouls"] = df_nba_stats["flagrantFouls"].astype(float)
df_nba_stats["playerId"] = df_nba_stats["playerId"].astype(int)

In [216]:
df_nba_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8164 entries, 0 to 8163
Data columns (total 39 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   playerId                          8164 non-null   int32  
 1   gamesPlayed                       8160 non-null   float64
 2   avgMinutes                        8160 non-null   float64
 3   avgFouls                          8164 non-null   float64
 4   flagrantFouls                     8164 non-null   float64
 5   technicalFouls                    8164 non-null   float64
 6   ejections                         8164 non-null   float64
 7   doubleDouble                      8164 non-null   float64
 8   tripleDouble                      8164 non-null   float64
 9   minutes                           8164 non-null   float64
 10  rebounds                          8164 non-null   float64
 11  fouls                             8164 non-null   float64
 12  avgReb

In [217]:
df_nba_experience.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8162 entries, 0 to 8161
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   playerId       8162 non-null   int64 
 1   salary         8162 non-null   object
 2   rankingSalary  8162 non-null   int64 
 3   experience     2304 non-null   object
dtypes: int64(2), object(2)
memory usage: 255.2+ KB


In [218]:
for idx, salary in enumerate(df_nba_experience["salary"]):
    if (type(salary) is str):
        value = salary.replace('$','').replace(',', '').strip()
        df_nba_experience.loc[idx, 'salary'] = value
        
df_nba_experience["salary"] = df_nba_experience["salary"].astype(float)

In [219]:
df_nba_experience.head()

,playerId,salary,rankingSalary,experience
0,366,9000000.0,15,None
1,366,10125000.0,27,None
2,366,11250000.0,17,None
3,366,12375000.0,14,None
4,366,13500000.0,11,None


In [220]:
df_nba_experience.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8162 entries, 0 to 8161
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   playerId       8162 non-null   int64  
 1   salary         8162 non-null   float64
 2   rankingSalary  8162 non-null   int64  
 3   experience     2304 non-null   object 
dtypes: float64(1), int64(2), object(1)
memory usage: 255.2+ KB


In [221]:
df_nba_season.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8162 entries, 0 to 8161
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   playerId    8162 non-null   int64 
 1   year        8162 non-null   int64 
 2   positionId  8156 non-null   object
 3   position    8162 non-null   object
 4   teamId      8162 non-null   object
 5   team        2304 non-null   object
 6   status      8162 non-null   object
dtypes: int64(2), object(5)
memory usage: 446.5+ KB


In [222]:
df_nba_season.head()

,playerId,year,positionId,position,teamId,team,status
0,366,1999,SG,Shooting Guard,PHI,NaN,inactive
1,366,2000,SG,Shooting Guard,PHI,NaN,inactive
2,366,2001,SG,Shooting Guard,PHI,NaN,inactive
3,366,2002,SG,Shooting Guard,PHI,NaN,inactive
4,366,2003,SG,Shooting Guard,PHI,NaN,inactive


In [223]:
os.mkdir('./processed')
df_nba_experience.to_csv("./processed/nba_experience.csv", index=False)
df_nba_personal.to_csv("./processed/nba_personal.csv", index=False)
df_nba_season.to_csv("./processed/nba_season.csv", index=False)
df_nba_stats.to_csv("./processed/nba_stats.csv", index=False)